In [1]:
import numpy as np
import sacc
import matplotlib.pyplot as plt
import pyccl as ccl
import scipy
import importlib as imp
import astropy.io.fits as fits

import CM_code.spurious_george as sp
imp.reload(sp)
import CM_code.halo_model as halo
imp.reload(halo)
import CM_code.lsst_coZmology as zed
imp.reload(zed)
import CM_code.tjp_helper as tjp_helper
imp.reload(tjp_helper)

# import main.py from TJPCov folder
import tjpcov as tjp
from tjpcov.covariance_calculator import CovarianceCalculator


plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['cmr10']
plt.rcParams['mathtext.fontset'] ='cm'
plt.rcParams['figure.facecolor'] = 'white'
plt.rc('axes', unicode_minus=False)
plt.rc('axes.formatter', use_mathtext=True)

### Y1 MEM Covaraince 

In [2]:
# create LSST sacc file
s = sacc.Sacc()

th_lsst = sp.theta_cents #forecasts

# start by adding redshift data (this will be used internally by TJP
# to create WL and NC tracers
z_l, dndz_l, zleff = zed.get_dndz_phot('lens', year=1)
s.add_tracer('NZ','lens', z=z_l, nz=dndz_l)

z_s, dndz_s, zseff = zed.get_dndz_phot('source', year=1)
s.add_tracer('NZ','source', z=z_s, nz=dndz_s)

A_IA = 0.16 * 5. * np.ones(len(z_l))

lensTracer = ccl.NumberCountsTracer(zed.cosmo_SRD, has_rsd=False, dndz=(z_l, dndz_l), 
                                    bias=(z_l, sp.lens_bias(z_l)))
shearTracer = ccl.WeakLensingTracer(zed.cosmo_SRD, (z_s, dndz_s), has_shear=True,
                                   ia_bias=(z_s, A_IA))

ell = np.unique(np.geomspace(180/(th_lsst[-1]/60.) - 10, 180/(th_lsst[0]/60.) + 10, 1024).astype(int))

Cl = ccl.angular_cl(zed.cosmo_SRD, lensTracer, shearTracer, ell)

LSST_xi = ccl.correlation(zed.cosmo_SRD, ell, Cl, th_lsst/60., type='NG') # take theta in degrees

s.add_theta_xi('galaxy_shearDensity_xi_t', 'lens', 'source', th_lsst, LSST_xi) # save in arcmins

s.save_fits('forecast_IA_data_lsst-y1.fits', overwrite=True)

In [5]:
rho = [0.1,0.9]

lsst_config = tjp_helper.generate_config(cosmo=zed.cosmo_SRD, sacc_file='forecast_IA_data_lsst-y1.fits',
                                   cov_type='RealGaussianFsky', Nlen=18., Nsrc=10., sig_e=0.26, len_bias=np.mean(sp.lens_bias(z_l)), IA=0.8,
                                   add_keys=[['ProjectedReal',['lmax', 42427]],['GaussianFsky',['fsky', 18000/41253]]])
LSST_covs = []
    
for value in rho:

    lsstX_config = tjp_helper.generate_config(cosmo=zed.cosmo_SRD, sacc_file='forecast_IA_data_lsst-y1.fits',
                                       cov_type='RealGaussianFsky', Nlen=18., Nsrc=10., sig_e=np.sqrt(value*0.26*0.26), 
                                        len_bias=np.mean(sp.lens_bias(z_l)), IA=0.8, 
                                        add_keys=[['ProjectedReal',['lmax', 42427]],['GaussianFsky',['fsky', 18000/41253]]])
    
    cov = CovarianceCalculator(lsst_config)
    cov_same = cov.get_covariance()

    cov = CovarianceCalculator(lsstX_config)
    cov_cross = cov.get_covariance()

    LSST_covs.append(2.*cov_same - 2.*cov_cross)
    
np.savez(file='generated_data/lsst-y1-covmats-IA', cov=LSST_covs, rho=rho)

Computing independent covariance blocks
('lens', 'source') ('lens', 'source')


/home/b7009348/WGL_project/LSST_forecast_code/tjpcov/covariance_builder.py:897: UserWarning: Missing n_ell_coupled info for some tracers in the sacc file. You will have to pass it withthe cache
  warnings.warn(


Building the covariance: placing blocks in their place
('lens', 'source') ('lens', 'source')
Computing independent covariance blocks
('lens', 'source') ('lens', 'source')
Building the covariance: placing blocks in their place
('lens', 'source') ('lens', 'source')
Computing independent covariance blocks
('lens', 'source') ('lens', 'source')
Building the covariance: placing blocks in their place
('lens', 'source') ('lens', 'source')
Computing independent covariance blocks
('lens', 'source') ('lens', 'source')
Building the covariance: placing blocks in their place
('lens', 'source') ('lens', 'source')


### Y10 MEM Covarainces

In [4]:
# create LSST sacc file
s = sacc.Sacc()

th_lsst = sp.theta_cents #forecasts

# start by adding redshift data (this will be used internally by TJP
# to create WL and NC tracers
z_l, dndz_l, zleff = zed.get_dndz_phot('lens', year=10)
s.add_tracer('NZ','lens', z=z_l, nz=dndz_l)

z_s, dndz_s, zseff = zed.get_dndz_phot('source', year=10)
s.add_tracer('NZ','source', z=z_s, nz=dndz_s)

A_IA = 0.16 * 5. * np.ones(len(z_l))

lensTracer = ccl.NumberCountsTracer(zed.cosmo_SRD, has_rsd=False, dndz=(z_l, dndz_l), 
                                    bias=(z_l, sp.lens_bias(z_l)))
shearTracer = ccl.WeakLensingTracer(zed.cosmo_SRD, (z_s, dndz_s), has_shear=True,
                                   ia_bias=(z_s, A_IA))

ell = np.unique(np.geomspace(180/(th_lsst[-1]/60.) - 10, 180/(th_lsst[0]/60.) + 10, 1024).astype(int))

Cl = ccl.angular_cl(zed.cosmo_SRD, lensTracer, shearTracer, ell)

LSST_xi = ccl.correlation(zed.cosmo_SRD, ell, Cl, th_lsst/60., type='NG') # take theta in degrees

s.add_theta_xi('galaxy_shearDensity_xi_t', 'lens', 'source', th_lsst, LSST_xi) # save in arcmins

s.save_fits('forecast_IA_data_lsst-y10.fits', overwrite=True)

In [6]:
rho = [0.1,0.9]

lsst_config = tjp_helper.generate_config(cosmo=zed.cosmo_SRD, sacc_file='forecast_IA_data_lsst-y10.fits',
                                   cov_type='RealGaussianFsky', Nlen=48., Nsrc=27., sig_e=0.13, len_bias=1.5, IA=0.8,
                                   add_keys=[['ProjectedReal',['lmax', 42427]],['GaussianFsky',['fsky', 18000/41253]]])
LSST_covs = []
    
for value in rho:

    lsstX_config = tjp_helper.generate_config(cosmo=zed.cosmo_SRD, sacc_file='forecast_IA_data_lsst-y10.fits',
                                       cov_type='RealGaussianFsky', Nlen=48., Nsrc=27., sig_e=np.sqrt(value*0.13*0.13), 
                                        len_bias=1.5, IA=0.8, 
                                        add_keys=[['ProjectedReal',['lmax', 42427]],['GaussianFsky',['fsky', 18000/41253]]])
    
    cov = CovarianceCalculator(lsst_config)
    cov_same = cov.get_covariance()

    cov = CovarianceCalculator(lsstX_config)
    cov_cross = cov.get_covariance()

    LSST_covs.append(2.*cov_same - 2.*cov_cross) 
    
np.savez(file='generated_data/lsst-y10-covmats-IA', cov=LSST_covs, rho=rho)

Computing independent covariance blocks
('lens', 'source') ('lens', 'source')
Building the covariance: placing blocks in their place
('lens', 'source') ('lens', 'source')
Computing independent covariance blocks
('lens', 'source') ('lens', 'source')
Building the covariance: placing blocks in their place
('lens', 'source') ('lens', 'source')
Computing independent covariance blocks
('lens', 'source') ('lens', 'source')
Building the covariance: placing blocks in their place
('lens', 'source') ('lens', 'source')
Computing independent covariance blocks
('lens', 'source') ('lens', 'source')
Building the covariance: placing blocks in their place
('lens', 'source') ('lens', 'source')
